In [1]:
import numpy as np
import librosa

In [2]:
code_list = [
 '1', '2', '3', 'A',
 '4', '5', '6', 'B',
 '7', '8', '9', 'C',
 '*', '0', '#', 'D',
]

# Coding frequences
cfs = [697, 770, 852, 941, 1209, 1336, 1477, 1633]

In [3]:
fs = 44100 # Sampling rate, Hz
t = 0.2 # Time interval, seconds
N = int(fs * t) # Number of samples for Goertzel's algo
Wn = 2*np.pi/N # Normalized frequency

alpha = lambda k: Wn*k

def goertzel(samples:list, freqs:list):
    #x0 = samples[0]
    samples = samples[-2::-1]
    magnitudes = dict()
    
    for f in freqs:
        #k = int(f*N/fs)
        c = 2*np.cos(2*np.pi*f/fs)
        
        g1, g2 = .0, .0
        for s in samples:
            g = s + c*g1 - g2
            g2 = g1
            g1 = g
        
        mag = np.sqrt(g2**2 + g1**2 - c*g1*g2)
        magnitudes[f] = mag
    
    return magnitudes

In [4]:
x, sr = librosa.load("DTMF_Lab6.wav", sr=fs)

threshold = 500

i = 0
while int(i*N) < len(x):
    samples = x[int(i*N):int(i*N) + N]
    magnitudes = goertzel(samples, cfs)
    code_freqs = [k for k, v in magnitudes.items() if v >= threshold]
    
    code_index = 4*cfs.index(min(code_freqs)) + (cfs.index(max(code_freqs)) - 4)
    print(code_list[code_index], end = " ")
    
    i += 1.5

1 A 2 * 3 B 4 # 5 C 6 D 7 * 8 # 